# The Merovingian

This Notebook is meant as a visual aid for users who wish to use the MAGNETOCOSMICS software to calculate Rigidity Cutoff values for primary cosmic rays. This Notebook makes use of [scripts developed by Mauricio Suárez Durán for his investigation works](https://github.com/suarez-duran-m/master-codes). This is *not* meant to replace those codes, but rather make their use easier for people who wish to employ them. The requirements are as follows:

* Python 3.6 or later
* Ubuntu OS
* The folder included in the GitHub page called *scripts*. This folder contains three files:
 * *wghour.d*, a data file containing all data for the Tsyganenko model for external geomagnetic field used.
 * *table-base.g4mac*, a simple text file containing the format used as an input for MAGNETOCOSMICS.
 * *maketable-magcos.sh*, an executable script that will create the macro input file for MAGCO using the parameters provided by the user.

It's important that the *scripts* folder be in the same location as this Notebook. Before running, we recommend that you read the Notebook in its **entirety**. To run, click *Cell* and *Run All*.

## Brief explanation of what this code does

The way we use MAGNETOCOSMICS in this work is meant to help us filter a list of primary cosmic rays from the CORSIKA software by the calculated values of Cutoff Rigidity ($R_C$). We use MAGNETOCOSMICS to calculate $R_C$ for multiple proton-like cosmic rays entering the upper atmosphere from multiple directions. For this, MAGNETOCOSMICS uses a MACRO file as an input where all the parameters desired by the user are found.

This Notebook will help the user to easily create said macro file, used as a MAGCO input, with all the desired parameters.

## The Code

### Block I: Input parameters

This section of the code asks the user for the required parameters for the calculation of the Cutoff Rigidity.

In [ ]:
# Libraries needed for the code
import datetime
import subprocess

This first cell asks the user for the time and location parameters for MAGNETOCOSMICS. The user must provide the latitude and longitude coordinates for the place of simulation in decimal degrees. The user must also provide the date information for the Tsyganenko model.

For the external geomagnetic field, MAGNETOCOSMICS makes use of the Tsyganenko model. The data for this model can be found in the file *scripts/wghour.d* and was taken from https://rdenton.host.dartmouth.edu/magpar/index.html. That database has geomagnetic data for the years 1963 to 2014, for every hour of every day. We ask the user to provide a specific date (day/month/year) as well as an hour of the day (integer from 0 to 23, with 0 being midnight and 23 being 11:00 pm). The earliest possible date is the 27th of November 1963 at 14 hours (2:00 pm); the latest possible date is the 23rd of March 2014 at 1 am. If the date+hour provided by the user is outside of these limits, the code will just use either of these two boundary values by default.

In [ ]:
# First we ask the user to provide the latitude and logitude for the simulation
print("""Please write the geographic coordinates of the place for simulation in decimal degrees.
For example, coordinates for the White House would be written like this:
Latitude: 38.90
Longitude: -77.04""")
lat = input("Latitude (dd.dd): ")
long = input("Longitude (dd.dd): ")

# Then the user must provide the date for the TSY01 model
print("""Now please provide the date (day, month and year) of simulation.
Data for the Tsyganenko model will be automatically extracted fron the datbase
based on the date and time information.
      """)
day, month, year = input("Date (dd/mm/yyyy): ").split("/")

# Finally the user must also provide the hour of the day for the TSY01 model
print("""Now please provide the time as an hour of the day. The value must be an INTEGER between 0 and 23.
Data for the Tsyganenko model will be automatically extracted fron the datbase
based on the date and time information.
      """)
hour = input("Time (0 - 23): ")

# All those parameters must be converted to either floats or integers
lat, long = float(lat), float(long)
hour, day, month, year = int(hour), int(day), int(month), int(year)

This cell below calculates where in the *wghour.d* datafile the required data can be found.

In [ ]:
# This part determines what line of the wghour.d file corresponds to the chosen date+time
startDate = datetime.datetime(1963,11,27,14)
chosenDate = datetime.datetime(year, month, day, hour)
dateDiff = chosenDate - startDate
hoursDiff = dateDiff.total_seconds()/3600
lineNumber = int(hoursDiff + 2)

# If the chosen date+time is outside limits, then the program will either choose the first or last line of the
# data file
if lineNumber < 2:
    lineNumber = 2
elif lineNumber > 441085:
    lineNumber = 441085

### Block II: Optional input parameters

MAGNETOCOSMICS can ask the user for a few other input parameters. Although these are necessary for the execution of MAGNETOCOSMICS, the user is not forced to provide them. We provide default values for these parameters and strongly suggest that the user stick to them.

The first of these parameters is the atmospheric altitude. This value is the height at which the simulated primary cosmic rays will enter the atmosphere. If the cell below is left unchanged, the default value for this will be 112.8 km, since this coincides with the maximum height of CORSIKA atmospheric profiles. If the user whishes to change this value, just **un-comment lines 7 and 8 (erase the symbol # in both lines)** in the cell below; with this, the program will ask the user to enter the desired altitude value.

In [ ]:
# This part asks the user to provide the altitude of the top of the atmosphere if desired
# For this, the user must un-comment lines 7 and 8.
altitude = 112.8
print("""The default value of altitude for the simulation is 112.8 km.
We strongly recommend you stick to this value. This value corresponds to the top of the atmosphere.
If you wish to change it, un-comment the code lines BELOW.""")
#altitude = input("Altitude (in km): ")
#altitude = float(altitude)

The other important values for the MAGNETOCOSMICS calculation are the *bins* for the incidence direction of the primary cosmic rays. The way MAGNETOCOSMICS works is that it calculates the Cutoff Rigidity values for particles entering the atmosphere at varied directions. This direction of incidence for these particles is given by zenith and azimuth angles.

If the cell below is left unchanged, then these will be the default values for the calculation:

* Zenith angle from 0° to 90° at 3° steps.
* Azimuth angle from 0° to 360° at 15° steps.

It's been proved that these default values hand out a solid result<sup>[[reference]](http://tangara.uis.edu.co/biblioweb/tesis/2020/178633.pdf)</sup>  and we stongly recommend that the user not change them. If the user whises to use different values, then just **un-comment all lines from 7 to 12 in the cell below (erase all the # symbols)**. Then, the code will ask the user to provide the desired *initial angle, final angle and steps* for both the zenith and azimuth angles.

In [ ]:
# This part asks the user to provide the range and steps for zenith and azimuth angles if desired
# For this, the user must un-comment lines 7 to 12.
a1, newB1, h1 = 0.0, 31, 3.0
a2, newB2, h2 = 0.0, 25, 15.0
print("""Un-comment the code lines below to change the default angle sweep values.
Take into account that (FinalAngle - InitialAngle) MUST be divisible by Step.""")
#a1, b1, h1 = input("Zenith angle sweep (InitialAngle,FinalAngle,Step)  (Max value = 90°): ").split(",")
#a2, b2, h2 = input("Zenith angle sweep (InitialAngle,FinalAngle,Step): (Max value = 360°)").split(",")
#a1, b1, h1 = float(a1), float(b1), float(h1)
#a2, b2, h2 = float(a2), float(b2), float(h2)
#newB1 = (b1-a1)/h1+1
#newB2 = (b2-a2)/h2+1

### Block III: Creation of the MACRO file

This last block will create the MACRO file with the given input parameters.

This first cell below will create a copy of the *table-base.g4mac* and will include in it the provided data from the TSY01 model, corresponding to the desired date and time.

In [ ]:
# This section will create a copy of the files in scripts/ in the current folder and will run the
# Executable bash script with the TSY01 chosen parameters to create the macro file
with open('scripts/maketable-magcos.sh', 'r') as file1:
    data1 = file1.readlines()
data1[3] = 'for i in $(seq '+str(lineNumber)+' '+str(lineNumber)+');do\n'
with open('table-copy.sh', 'w') as file2:
    file2.writelines(data1)
execu = subprocess.run(["chmod", "+x", "scripts/table-copy.sh"])
cp1 = subprocess.run(["cp", "scripts/table-base.g4mac", "."])
cp2 = subprocess.run(["cp", "scripts/wghour.d", "."])
run = subprocess.run(["bash", "table-copy.sh"])

# This part delates temporal and/or unnecessary files.
erase1 = subprocess.run(["rm", "table-copy.sh"])
erase2 = subprocess.run(["rm", "wghour.d"])
erase3 = subprocess.run(["rm", "table-base.g4mac"])

The scripts here used will name this copy based on the date and name provided. This cell belows determines what the name of the file is.

In [ ]:
# The new macro file has been created with a name corresponding to the year+day+hour chosen by the user.
# This part of the code determines exactly what the name of the macro is, so it can be edited in the next cell.
with open('scripts/wghour.d', 'r') as dataFile:
    newTimeData = dataFile.readlines()
lista = newTimeData[lineNumber-1].split(' ')
newNewTimeData = [l for l in lista if l != '']
fileYear = newNewTimeData[0]
fileDay = newNewTimeData[1]
fileHour = newNewTimeData[2]

This last cell will write the rest of the important data onto the newly created file. This data incudes coordinates, altitude, and values for the zenith and azimuth angles.

In [ ]:
# This last part opens the macro file and edits it to include the remaining information,
# i. e. altitude, coordinates, zenith and azimuth angles ranges and steps.
macroName = 'maketable-'+str(fileYear)+'-d'+str(fileDay)+'-h'+str(fileHour)+'.g4mac'
with open(macroName, 'r') as macro:
    macroData = macro.readlines()
macroData[-3] = '/MAGCOS/SOURCE/SetPosition GEO '+str(altitude)+' km '+str(lat)+' '+str(long)+' degree # BRC\n'
macroData[-1] = '/MAGCOS/SCENARIO/RCutoffVsDirection GEO '+str(a1)+' '+str(h1)+' '+str(newB1)+' '+str(a2)+' '+str(h2)+' '+str(newB2)+' '+str(macroName)+'\n'
with open(macroName, 'w') as finalMacro:
    finalMacro.writelines(macroData)

Finally, the result of this code is a MACRO file, called maketable-**year**-d**day**-h**hour**.g4mac. Here, the words **year**, **day** and **hour** in bold will correspond to the values chosen by the user. This file has to be used as input for MAGNETOCOSMICS.

## How do we use the created MACRO file?

The file maketable-**year**-d**day**-h**hour**.g4mac that's created at the end of this code can be used to run MAGNETOCOSMICS. The command to run it is:

```shell
STATIC_MAGNETOCOSMICS maketable-yyyy-ddd-hhh.g4mac
```
If you have MAGNETOCOSMICS and all of its requirementes installed in your computer, then you can run that line in the Ubuntu terminal to execute MAGNETOCOSMICS. However, this is something that consumes a lot of computational resources and time, so we strongly suggest you run MAGNETOCOSMICS in a remote server / cluster so you don't have to worry abount the computational cost.

Whatever method the user chooses, the result **after** MAGNETOCOSMICS has been run will be a data text file contaning ALL the calculated $R_C$ values for incidence directions corresponding to the user-selected zenith and azimuth angles. That data file will be used combined with CRKtools to filter CORSIKA output files, so the effect of the geomagnetic field can be taken into account for secondary cosmic rays atmospheric showers.